# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [2]:
import os
import sys

sys.path.append('..')
from utils import *

# Set your preferred name
USER = "Arthur"
# Set the data and output directories
DATA_ROOT = 'C:/Users/arthu/Downloads'
OUTPUT_ROOT = 'C:/Users/arthu/OneDrive/Documents/GitHub/output2'
TRAIT = 'Longevity-and-Aging'
OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

utils.py has been loaded


In [3]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Cervical_Cancer_(CESC)']

If no match is found, jump directly to GEO in Part 2.2

In [27]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [28]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.639586,-0.195527,-0.324695,0.193167,-0.572812
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.997486,-0.276027,-1.219395,1.850967,-0.471212
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.959514,0.714373,0.112305,-0.532833,-0.003812
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,1.013722,-1.587117,0.359267,2.59051,0.806623,0.812114,0.367373,-0.494595,-1.062233,0.108588
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,0.110214,1.469373,0.194605,1.404667,-0.290712


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lung-Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Lung-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [3]:
dataset = 'GEO'
trait_subdir = "Longevity-and-Aging"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE155177', 'GSE16717', 'GSE234154', 'GSE44147', 'GSE4741', 'GSE48264']

Repeat the below steps for all the accession numbers

In [21]:
cohort = accession_num = "GSE48264"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('C:/Users/arthu/Downloads\\GEO\\Longevity-and-Aging\\GSE48264\\GSE48264_family.soft.gz',
 'C:/Users/arthu/Downloads\\GEO\\Longevity-and-Aging\\GSE48264\\GSE48264_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [22]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Uppsala Longitudinal Study of Adult Men (ULSAM)"
!Series_summary	"The Uppsala Longitudinal Study of Adult Men is a population-based study aimed at identifying risk factors for cardiovascular disease. At the time of biopsy all subjects were ~ 70yr of age"
!Series_overall_design	"We extracted RNA from muscle tissue taken from 129 subjects, when they were aged ~70yr and characterised as disease-free (note the above average longevity in Swedes born circa 1920 compared with US and UK populations). From these samples, 108 yielded RNA of sufficient quality to profile on Affymetrix gene-chips."
!Series_overall_design	"Only survival data are used in the paper."
!Series_overall_design	"There are no data from cardiovascular disease subjects; we only profiled the healthy subjects and followed for 20yrs."


In [23]:
clinical_data.head()

,!Sample_geo_accession,GSM1173505,GSM1173506,GSM1173507,GSM1173508,GSM1173509,GSM1173510,GSM1173511,GSM1173512,GSM1173513,...,GSM1173603,GSM1173604,GSM1173605,GSM1173606,GSM1173607,GSM1173608,GSM1173609,GSM1173610,GSM1173611,GSM1173612
0,!Sample_characteristics_ch1,disease-free: disease-free,disease-free: disease-free,disease-free: disease-free,disease-free: disease-free,disease-free: disease-free,disease-free: disease-free,disease-free: disease-free,disease-free: disease-free,disease-free: disease-free,...,disease-free: disease-free,disease-free: disease-free,disease-free: disease-free,disease-free: disease-free,disease-free: disease-free,disease-free: disease-free,disease-free: disease-free,disease-free: disease-free,disease-free: disease-free,disease-free: disease-free
1,!Sample_characteristics_ch1,age(approx): 70 yr,age(approx): 70 yr,age(approx): 70 yr,age(approx): 70 yr,age(approx): 70 yr,age(approx): 70 yr,age(approx): 70 yr,age(approx): 70 yr,age(approx): 70 yr,...,age(approx): 70 yr,age(approx): 70 yr,age(approx): 70 yr,age(approx): 70 yr,age(approx): 70 yr,age(approx): 70 yr,age(approx): 70 yr,age(approx): 70 yr,age(approx): 70 yr,age(approx): 70 yr
2,!Sample_characteristics_ch1,tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),...,tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline),tissue: skeletal muscle biopsy (baseline)
3,!Sample_characteristics_ch1,survival: None,survival: Hosp,survival: Death,survival: Hosp,survival: Hosp,survival: Death,survival: None,survival: Death,survival: Hosp,...,survival: Hosp,survival: Death,survival: Hosp,survival: Death,survival: Hosp,survival: Hosp,survival: Hosp,survival: Hosp,survival: Hosp,survival: None
4,!Sample_characteristics_ch1,patient id: 32,patient id: 117,patient id: 152,patient id: 211,patient id: 241,patient id: 254,patient id: 255,patient id: 296,patient id: 298,...,patient id: 1573,patient id: 1597,patient id: 1609,patient id: 1617,patient id: 1631,patient id: 1638,patient id: 1649,patient id: 1655,patient id: 1661,patient id: 1670


In [24]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['disease-free: disease-free'],
 1: ['age(approx): 70 yr'],
 2: ['tissue: skeletal muscle biopsy (baseline)'],
 3: ['survival: None', 'survival: Hosp', 'survival: Death'],
 4: ['patient id: 32',
  'patient id: 117',
  'patient id: 152',
  'patient id: 211',
  'patient id: 241',
  'patient id: 254',
  'patient id: 255',
  'patient id: 296',
  'patient id: 298',
  'patient id: 300',
  'patient id: 317',
  'patient id: 349',
  'patient id: 351',
  'patient id: 355',
  'patient id: 373',
  'patient id: 377',
  'patient id: 381',
  'patient id: 397',
  'patient id: 421',
  'patient id: 465',
  'patient id: 498',
  'patient id: 521',
  'patient id: 549',
  'patient id: 554',
  'patient id: 576',
  'patient id: 621',
  'patient id: 632',
  'patient id: 634',
  'patient id: 664',
  'patient id: 674']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [25]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Longevity-and-Aging\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Longevity-and-Aging\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. 

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [26]:
is_gene_availabe =  True
trait_row = None
age_row = 1
gender_row = None

trait_type = None

In [27]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

This cohort is not usable. Please skip the following steps and jump to the next accession number.


In [10]:
def convert_trait(value):
    if value in ['group: long-lived sib', 'group: control', 'group: offspring']:
        return 1  # Representing presence of the trait
    else:
        return 0  # Representing absence of the trait or unknown value


def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            #return int(age_string.split('age: ')[1].rstrip('years'))  # Extracting numeric part
            return float(age_string.split('age: ')[1].rstrip(' years'))  
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format
def convert_gender(value):
    if value in ['gender: female', 'gender: male']:
        return value.split(' ')[1]  # Extracting 'female' or 'male'
    else:
        return None  # Handling unknown or incorrectly formatted values


# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23.1 years'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(0, 23.1, None)

In [11]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM418770,GSM418771,GSM418772,GSM418773,GSM418774,GSM418775,GSM418776,GSM418777,GSM418778,GSM418779,...,GSM418910,GSM418911,GSM418912,GSM418913,GSM418914,GSM418915,GSM418916,GSM418917,GSM418918,GSM418919
Longevity-and-Aging,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Age,91.53,56.1,91.52,52.83,64.11,64.27,59.75,93.4,61.47,93.19,...,66.29,43.71,42.79,91.62,63.92,97.14,66.85,68.17,92.69,94.95
Gender,female,female,male,male,female,male,male,female,male,male,...,male,female,male,male,male,female,male,male,female,male


### Genetic data preprocessing and final filtering

In [12]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM418770,GSM418771,GSM418772,GSM418773,GSM418774,GSM418775,GSM418776,GSM418777,GSM418778,GSM418779,...,GSM418910,GSM418911,GSM418912,GSM418913,GSM418914,GSM418915,GSM418916,GSM418917,GSM418918,GSM418919
ID,,,,,,,,,,,,,,,,,,,,,
1005,3.270627,1.443745,2.165435,2.958053,-0.012361,0.183089,4.678742,1.355026,0.476758,0.576805,...,1.166188,2.421311,3.554502,3.178734,3.357477,2.879241,2.383955,2.694113,2.528392,0.731126
1006,3.357982,3.001623,2.098785,1.874185,3.126547,2.542565,2.249156,4.621413,4.535040,2.425851,...,4.311294,2.552124,3.938536,3.109024,3.652286,3.989664,3.616673,3.478957,2.152819,3.623570
1007,2.850012,3.756270,-0.479744,3.997663,4.377559,3.540897,4.449276,2.865519,3.787468,5.486711,...,3.867406,2.439018,3.067604,3.370675,3.998063,3.350959,4.193385,3.841488,3.893967,3.375831
1010,11.689739,11.289610,11.361654,11.010983,11.077953,11.207382,11.329192,11.387695,11.221407,11.102112,...,11.685822,11.431046,11.653290,11.127514,11.684558,11.465079,11.531564,10.768201,11.167387,11.428991
1011,5.379569,5.197754,4.909275,5.078186,4.634247,4.722292,5.426901,5.420935,5.072376,5.073200,...,5.243837,5.171475,5.528213,5.239949,5.522672,5.542548,5.593226,5.039504,5.343694,5.227207


In [13]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['1005',
 '1006',
 '1007',
 '1010',
 '1011',
 '1014',
 '1023',
 '1027',
 '1028',
 '1048',
 '1054',
 '1059',
 '1061',
 '1065',
 '1072',
 '1073',
 '1074',
 '1076',
 '1079',
 '1080']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [14]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1005', '1006', '1007', '1010', '1011', '1014', '1023', '1027', '1028', '1048', '1054', '1059', '1061', '1065', '1072', '1073', '1074', '1076', '1079', '1080']\n"

If not required, jump directly to the gene normalization step

In [15]:
requires_gene_mapping = True

In [16]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

: 

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [27]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['MIMAT0000062', 'MIMAT0000063', 'MIMAT0000064', 'MIMAT0000065', 'MIMAT0000066'], 'miRNA': ['MIMAT0000062', 'MIMAT0000063', 'MIMAT0000064', 'MIMAT0000065', 'MIMAT0000066'], 'miRNA_ID_LIST': ['hsa-let-7a-5p', 'hsa-let-7b-5p', 'hsa-let-7c-5p', 'hsa-let-7d-5p', 'hsa-let-7e-5p']}
    


In [28]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key ='GB_LIST'

    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

KeyError: "['GB_LIST'] not in index"

In [29]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [30]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [31]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 0 samples.


In [32]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

ValueError: attempt to get argmin of an empty sequence

In [33]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

NameError: name 'is_trait_biased' is not defined

In [34]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

NameError: name 'is_trait_biased' is not defined

### 3. Do regression & Cross Validation

In [35]:
from utils import *

In [36]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jacob/Documents/output\\Jake\\Longevity-and-Aging\\cohort_info.json'

In [37]:
ranked_df.head()

NameError: name 'ranked_df' is not defined

In [51]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [52]:
merged_data.head()

,Longevity-and-Aging,Age
0,1.0,91.53
1,0.0,56.10
2,0.0,64.11
3,1.0,93.40
4,0.0,53.40


In [53]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [54]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [55]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

False

In [57]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [58]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

ValueError: Found array with 0 feature(s) (shape=(60, 0)) while a minimum of 1 is required by Lasso.

In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

### 5. Discussion and report

In [ ]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0025
Found 547 significant genes associated with the trait 'Psoriasis' conditional on the factor 'Age', with corrected p-value < 0.05:
   Variable  Coefficient  corrected_p_value
4.430015695     0.087750           0.001231
4.478193647     0.087750           0.001231
4.478966158     0.087750           0.001231
 4.48687579     0.087750           0.001231
4.491851159     0.087750           0.001231
4.492453853     0.087750           0.001231
4.494270678     0.087750           0.001231
4.504245863     0.087750           0.001231
4.505190735     0.087750           0.001231
4.505235806     0.087750           0.001231
4.506891502     0.087750           0.001231
4.510669146     0.087750           0.001231
4.476613942     0.087750           0.001231
 4.51198966     0.087750           0.001231
4.513528153     0.087750           0.001231
4.516930407     0.087750           0.001231
4.806825082     0.087750           0.001

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values


In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)